# Transfer Learning CIFAR10

* Train a simple convnet on the CIFAR dataset the first 5 output classes [0..4].
* Freeze convolutional layers and fine-tune dense layers for the last 5 ouput classes [5..9].


### 1. Import CIFAR10 data and create 2 datasets with one dataset having classes from 0 to 4 and other having classes from 5 to 9 

In [0]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
%tensorflow_version 2.x
import tensorflow as tf

In [0]:
(X_train, y_train),(X_test, y_test) = tf.keras.datasets.cifar10.load_data()

In [0]:
X_train_0_to_4 = X_train[y_train[:,0]<5]
X_test_0_to_4 = X_test[y_test[:,0]<5]
y_train_0_to_4 = y_train[y_train[:,0]<5]
y_test_0_to_4 = y_test[y_test[:,0]<5]

X_train_5_to_9 = X_train[y_train[:,0]>4]
X_test_5_to_9 = X_test[y_test[:,0]>4]
y_train_5_to_9 = y_train[y_train[:,0]>4]
y_test_5_to_9 = y_test[y_test[:,0]>4]

In [20]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(50000, 32, 32, 3)
(10000, 32, 32, 3)
(50000, 1)
(10000, 1)


### 2. Use One-hot encoding to divide y_train and y_test into required no of output classes

In [0]:
y_train_cat_0_to_4 = tf.keras.utils.to_categorical(y_train_0_to_4, num_classes=10)
y_test_cat_0_to_4 = tf.keras.utils.to_categorical(y_test_0_to_4, num_classes=10)
y_train_cat_5_to_9 = tf.keras.utils.to_categorical(y_train_5_to_9, num_classes=10)
y_test_cat_5_to_9 = tf.keras.utils.to_categorical(y_test_5_to_9, num_classes=10)

### 3. Build a sequential neural network model which can classify the classes 0 to 4 of CIFAR10 dataset with at least 80% accuracy on test data

In [0]:
X_train_norm_0_to_4 = X_train_0_to_4/np.float32(255)
X_train_norm_5_to_9 = X_train_5_to_9/np.float32(255)
X_test_norm_0_to_4 = X_test_0_to_4/np.float32(255)
X_test_norm_5_to_9 = X_test_5_to_9/np.float32(255)

In [0]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.backend import clear_session
from tensorflow.keras.layers import Reshape, Dense, Dropout, BatchNormalization, Activation, Conv2D, MaxPool2D, Flatten, GlobalAveragePooling2D

In [0]:
tf.keras.backend.clear_session()

#Initialize model
model = Sequential()

#normalize data
model.add(BatchNormalization(input_shape=(32,32,3,)))

#Add Conv Layer
model.add(Conv2D(64, kernel_size=(3,3), activation='relu'))

#normalize data
model.add(BatchNormalization(input_shape=(32,32,3,)))

#Add Max Pool layer
model.add(MaxPool2D(pool_size=(2,2)))

#Add Conv Layer
model.add(Conv2D(64, kernel_size=(3,3), activation='relu'))

#normalize data
model.add(BatchNormalization(input_shape=(32,32,3,)))

#Add Max Pool layer
model.add(MaxPool2D(pool_size=(2,2)))

#Add Dense Layers after flattening the data
model.add(Flatten())

#Add Dropout
model.add(Dropout(0.25))

#Add Output Layer
model.add(Dense(10, activation='softmax'))

In [0]:
#Specify Loass and Optimizer
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [0]:
from keras.callbacks import ModelCheckpoint

checkpointer = ModelCheckpoint(filepath='best_model.hdf5',  verbose=1, save_best_only=True)

In [61]:
model.fit(X_train_norm_0_to_4,y_train_cat_0_to_4, validation_data=(X_test_norm_0_to_4,y_test_cat_0_to_4), epochs=10, batch_size=32, verbose=1, callbacks=[checkpointer], shuffle=True)

Train on 25000 samples, validate on 5000 samples
Epoch 1/10
24768/25000 [============================>.] - ETA: 0s - loss: 0.1949 - accuracy: 0.9213
Epoch 00001: val_loss improved from inf to 0.16034, saving model to best_model.hdf5
25000/25000 [==============================] - 5s 197us/sample - loss: 0.1946 - accuracy: 0.9215 - val_loss: 0.1603 - val_accuracy: 0.9353
Epoch 2/10
24960/25000 [============================>.] - ETA: 0s - loss: 0.1451 - accuracy: 0.9396
Epoch 00002: val_loss improved from 0.16034 to 0.12732, saving model to best_model.hdf5
25000/25000 [==============================] - 4s 168us/sample - loss: 0.1451 - accuracy: 0.9396 - val_loss: 0.1273 - val_accuracy: 0.9464
Epoch 3/10
24864/25000 [============================>.] - ETA: 0s - loss: 0.1269 - accuracy: 0.9479
Epoch 00003: val_loss improved from 0.12732 to 0.12616, saving model to best_model.hdf5
25000/25000 [==============================] - 4s 169us/sample - loss: 0.1270 - accuracy: 0.9479 - val_loss: 0.12

In [0]:
initial_weights = model.get_weights()

### 4. In the model which was built above (for classification of classes 0-4 in CIFAR10), make only the dense layers to be trainable and conv layers to be non-trainable

In [0]:
tran_model = model

In [0]:
tran_model.set_weights(initial_weights)

In [67]:
tran_model.layers

In [69]:
tran_model.layers[9]

In [71]:
#Set all layers trainable to False (except dense layer)
for layer in tran_model.layers:
    layer.trainable = False
tran_model.layers[9].trainable = True
print(tran_model.layers[9])

In [75]:
tran_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization (BatchNo (None, 32, 32, 3)         12        
_________________________________________________________________
conv2d (Conv2D)              (None, 30, 30, 64)        1792      
_________________________________________________________________
batch_normalization_1 (Batch (None, 30, 30, 64)        256       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 15, 15, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 13, 13, 64)        36928     
_________________________________________________________________
batch_normalization_2 (Batch (None, 13, 13, 64)        256       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 6, 6, 64)          0

### 5. Utilize the the model trained on CIFAR 10 (classes 0 to 4) to classify the classes 5 to 9 of CIFAR 10  (Use Transfer Learning) <br>
Achieve an accuracy of more than 85% on test data

In [0]:
#Compile model
tran_model.compile(loss="binary_crossentropy", optimizer='adam',metrics=["accuracy"])

In [73]:
tran_model.fit(X_test_norm_5_to_9, y_test_cat_5_to_9, epochs=10, batch_size=32, verbose=1, callbacks=[checkpointer], shuffle=True)

Train on 5000 samples
Epoch 1/10
5000/5000 [==============================] - 1s 191us/sample - loss: 0.4071 - accuracy: 0.9083
Epoch 2/10
5000/5000 [==============================] - 1s 117us/sample - loss: 0.0972 - accuracy: 0.9654
Epoch 3/10
5000/5000 [==============================] - 1s 112us/sample - loss: 0.0730 - accuracy: 0.9732
Epoch 4/10
5000/5000 [==============================] - 1s 119us/sample - loss: 0.0622 - accuracy: 0.9773
Epoch 5/10
5000/5000 [==============================] - 1s 110us/sample - loss: 0.0535 - accuracy: 0.9801
Epoch 6/10
5000/5000 [==============================] - 1s 114us/sample - loss: 0.0469 - accuracy: 0.9829
Epoch 7/10
5000/5000 [==============================] - 1s 123us/sample - loss: 0.0446 - accuracy: 0.9832
Epoch 8/10
5000/5000 [==============================] - 1s 120us/sample - loss: 0.0409 - accuracy: 0.9854
Epoch 9/10
5000/5000 [==============================] - 1s 116us/sample - loss: 0.0385 - accuracy: 0.9863
Epoch 10/10
5000/5000 [=

# Text classification using TF-IDF

### 6. Load the dataset from sklearn.datasets

In [0]:
from sklearn.datasets import fetch_20newsgroups

In [0]:
categories = ['alt.atheism', 'soc.religion.christian', 'comp.graphics', 'sci.med']

### 7. Training data

In [0]:
twenty_train = fetch_20newsgroups(subset='train', categories=categories, shuffle=True, random_state=42)

### 8. Test data

In [0]:
twenty_test = fetch_20newsgroups(subset='test', categories=categories, shuffle=True, random_state=42)

###  a.  You can access the values for the target variable using .target attribute 
###  b. You can access the name of the class in the target variable with .target_names


In [0]:
twenty_train.target

array([1, 1, 3, ..., 2, 2, 2])

In [0]:
twenty_train.target_names

['alt.atheism', 'comp.graphics', 'sci.med', 'soc.religion.christian']

In [0]:
twenty_train.data[0:5]

['From: sd345@city.ac.uk (Michael Collier)\nSubject: Converting images to HP LaserJet III?\nNntp-Posting-Host: hampton\nOrganization: The City University\nLines: 14\n\nDoes anyone know of a good way (standard PC application/PD utility) to\nconvert tif/img/tga files into LaserJet III format.  We would also like to\ndo the same, converting to HPGL (HP plotter) files.\n\nPlease email any response.\n\nIs this the correct group?\n\nThanks in advance.  Michael.\n-- \nMichael Collier (Programmer)                 The Computer Unit,\nEmail: M.P.Collier@uk.ac.city                The City University,\nTel: 071 477-8000 x3769                      London,\nFax: 071 477-8565                            EC1V 0HB.\n',
 "From: ani@ms.uky.edu (Aniruddha B. Deglurkar)\nSubject: help: Splitting a trimming region along a mesh \nOrganization: University Of Kentucky, Dept. of Math Sciences\nLines: 28\n\n\n\n\tHi,\n\n\tI have a problem, I hope some of the 'gurus' can help me solve.\n\n\tBackground of the probl

### 9.  Now with dependent and independent data available for both train and test datasets, using TfidfVectorizer fit and transform the training data and test data and get the tfidf features for both

In [0]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
vect = TfidfVectorizer()
X_train = pd.DataFrame(vect.fit_transform(twenty_train.data[:]).toarray(), columns=vect.get_feature_names())

In [0]:
y_train = twenty_train.target

In [0]:
X_test = pd.DataFrame(vect.transform(twenty_test.data[:]).toarray(), columns=vect.get_feature_names())

In [0]:
y_test = twenty_test.target

In [0]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(2257, 35788)
(1502, 35788)
(2257,)
(1502,)


### 10. Use logisticRegression with tfidf features as input and targets as output and train the model and report the train and test accuracy score

In [0]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
logreg = LogisticRegression()
logreg.fit(X_train, y_train)
y_pred_class = logreg.predict(X_test)
print("Training Score: " , logreg.score(X_train, y_train))
print("Testing Score: " , logreg.score(X_test, y_test))

Training Score:  0.9920248116969429
Testing Score:  0.8974700399467377
